In [1]:
%pip install llmcompressor
%pip install -U llmcompressor transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.1/282.1 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.6/192.6 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 144.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 51.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-ml-py
    Found existing installation: nvidia-ml-py 13.590.48
    Uninstalling nvidia-ml-py-13.590.48:
      Successfully uninstalled nvidia-ml-py-13.590.48
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.67.3
    Uninsta

In [2]:
import os
import json
import re
import hashlib
import torch
import shutil
from pathlib import Path
from datetime import datetime, timezone

from google.colab import drive
drive.mount('/content/drive')

from datasets import Dataset, concatenate_datasets, load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier


Mounted at /content/drive


In [3]:
MODEL_ID = "/content/drive/MyDrive/LGAimers/base_model"

NUM_CALIBRATION_SAMPLES = 1024
MAX_SEQUENCE_LENGTH = 512
CALIBRATION_SEED = 42

CALIB_DATASETS = [
    {
        "id": "LGAI-EXAONE/MANTA-1M",
        "split": "train",
        "n_samples": 512,
        "format": "manta",
        "priority": 1,
    },
    {
        "id": "nlpai-lab/kullm-v2",
        "split": "train",
        "n_samples": 256,
        "format": "instruction",
        "priority": 2,
    },
    {
        "id": "heegyu/OIG-small-chip2-ko",
        "split": "train",
        "n_samples": 192,
        "format": "oig_human_bot",
        "priority": 3,
    },
    {
        "id": "beomi/KoAlpaca-v1.1a",
        "split": "train",
        "n_samples": 64,
        "format": "instruction",
        "priority": 4,
    },
]

CALIBRATION_BENCHMARK_EXCLUDE = {
    "LGAI-EXAONE/KoMT-Bench",
    "LGAI-EXAONE/KMMLU-Redux",
    "LGAI-EXAONE/KMMLU-Pro",
}
KOALPACA_FALLBACK_ID = "nlpai-lab/kullm-v2"

assert sum(spec["n_samples"] for spec in CALIB_DATASETS) == NUM_CALIBRATION_SAMPLES, (
    "CALIB_DATASETS 샘플 합계와 NUM_CALIBRATION_SAMPLES가 일치해야 합니다."
)
assert not any(spec["id"] in CALIBRATION_BENCHMARK_EXCLUDE for spec in CALIB_DATASETS), (
    "평가용 벤치마크 데이터셋은 calibration에서 제외해야 합니다."
)

PRIMARY_SCHEME = "W4A16"
FALLBACK_SCHEME = "W8A8"

ATTN_TARGETS = [
    "re:.*self_attn\\.q_proj",
    "re:.*self_attn\\.k_proj",
    "re:.*self_attn\\.v_proj",
    "re:.*self_attn\\.o_proj",
]
IGNORE = ["embed_tokens", "lm_head"]

OUT_DIR = "/content/drive/MyDrive/LGAimers/sq_w4a16_attn_calmix1024"


In [4]:
print("[INFO] 모델 로드 중...")

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.bfloat16,
)

print("[INFO] 모델/토크나이저 로드 완료")

[INFO] 모델 로드 중...


`torch_dtype` is deprecated! Use `dtype` instead!


[INFO] 모델/토크나이저 로드 완료


In [5]:
import re


def collect_target_modules(model, target_patterns):
    all_module_names = [name for name, _ in model.named_modules()]

    matched_by_pattern = {}
    for pattern in target_patterns:
        if pattern.startswith("re:"):
            regex = pattern[len("re:"):]
        else:
            regex = re.escape(pattern)
        matches = [name for name in all_module_names if re.fullmatch(regex, name)]
        matched_by_pattern[pattern] = matches
        print(f"[TARGET] pattern={pattern} matches={len(matches)} sample={matches[:3]}")

    unmatched_patterns = [p for p, m in matched_by_pattern.items() if not m]
    if unmatched_patterns:
        raise AssertionError(f"No module matched for patterns: {unmatched_patterns}")

    unique_matches = sorted({m for matches in matched_by_pattern.values() for m in matches})
    mlp_overlap = [name for name in unique_matches if ".mlp." in name]
    if mlp_overlap:
        raise AssertionError(f"MLP modules were matched unexpectedly: {mlp_overlap[:10]}")

    expected_suffixes = {
        "self_attn.q_proj",
        "self_attn.k_proj",
        "self_attn.v_proj",
        "self_attn.o_proj",
    }
    per_layer = {}
    for name in unique_matches:
        parts = name.split(".")
        if len(parts) < 5 or parts[0] != "model" or parts[1] != "layers" or not parts[2].isdigit():
            continue
        layer_id = int(parts[2])
        suffix = ".".join(parts[3:])
        per_layer.setdefault(layer_id, set()).add(suffix)

    missing_per_layer = {
        layer_id: sorted(expected_suffixes - suffixes)
        for layer_id, suffixes in per_layer.items()
        if suffixes != expected_suffixes
    }
    if missing_per_layer:
        raise AssertionError(f"Incomplete attention targets per layer: {missing_per_layer}")

    expected_total = 120
    if len(unique_matches) != expected_total:
        raise AssertionError(
            f"Expected {expected_total} attention modules (30 layers x 4 projections), got {len(unique_matches)}"
        )

    layer_ids = sorted(per_layer)
    print(f"[TARGET] total_unique_matches={len(unique_matches)}")
    print(f"[TARGET] layer_count={len(layer_ids)}, layer_range=({layer_ids[0]}, {layer_ids[-1]})")

    return unique_matches


ATTN_MATCHED_MODULES = collect_target_modules(model, ATTN_TARGETS)
print("[INFO] attention-only target validation complete")


[TARGET] pattern=re:.*self_attn\.q_proj matches=30 sample=['model.layers.0.self_attn.q_proj', 'model.layers.1.self_attn.q_proj', 'model.layers.2.self_attn.q_proj']
[TARGET] pattern=re:.*self_attn\.k_proj matches=30 sample=['model.layers.0.self_attn.k_proj', 'model.layers.1.self_attn.k_proj', 'model.layers.2.self_attn.k_proj']
[TARGET] pattern=re:.*self_attn\.v_proj matches=30 sample=['model.layers.0.self_attn.v_proj', 'model.layers.1.self_attn.v_proj', 'model.layers.2.self_attn.v_proj']
[TARGET] pattern=re:.*self_attn\.o_proj matches=30 sample=['model.layers.0.self_attn.o_proj', 'model.layers.1.self_attn.o_proj', 'model.layers.2.self_attn.o_proj']
[TARGET] total_unique_matches=120
[TARGET] layer_count=30, layer_range=(0, 29)
[INFO] attention-only target validation complete


In [7]:
print("[INFO] 캘리브레이션 데이터 로드 중...")

def _clean_text(value):
    if value is None:
        return ""
    return str(value).strip()

def _normalize_manta_conversations(conversations):
    if not isinstance(conversations, list):
        return None

    normalized = []
    for turn in conversations:
        if not isinstance(turn, dict):
            continue
        role = _clean_text(turn.get("role")).lower()
        content = _clean_text(turn.get("content"))
        if role in {"system", "user", "assistant"} and content:
            normalized.append({"role": role, "content": content})

    has_user = any(turn["role"] == "user" for turn in normalized)
    has_assistant = any(turn["role"] == "assistant" for turn in normalized)
    if not (has_user and has_assistant):
        return None
    return normalized

def _normalize_instruction_turn(example):
    instruction = _clean_text(example.get("instruction"))
    input_text = _clean_text(example.get("input"))
    output_text = _clean_text(example.get("output"))

    user_content = instruction
    if input_text:
        user_content = f"{instruction}\n\n{input_text}" if instruction else input_text

    if not user_content or not output_text:
        return None

    return [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": output_text},
    ]

def _normalize_oig_pair(example):
    # heegyu/OIG-small-chip2-ko는 user/chip2 또는 *_translated 컬럼 기반이다.
    user_content = _clean_text(example.get("user_translated")) or _clean_text(example.get("user"))
    assistant_content = _clean_text(example.get("chip2_translated")) or _clean_text(example.get("chip2"))
    if user_content and assistant_content:
        return [
            {"role": "user", "content": user_content},
            {"role": "assistant", "content": assistant_content},
        ]

    # 구버전/파생셋 호환: text 컬럼에 <human>/<bot> 태그가 있는 경우 파싱.
    raw = _clean_text(example.get("text"))
    if not raw:
        return None

    pattern = re.compile(r"<human>\s*(.*?)\s*<bot>\s*(.*?)(?=<human>|$)", re.IGNORECASE | re.DOTALL)
    match = pattern.search(raw)
    if not match:
        return None

    user_content = _clean_text(match.group(1))
    assistant_content = _clean_text(match.group(2))
    if not user_content or not assistant_content:
        return None

    return [
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": assistant_content},
    ]

def normalize_example(example, source_format):
    if source_format == "manta":
        conversations = _normalize_manta_conversations(example.get("conversations"))
    elif source_format == "instruction":
        conversations = _normalize_instruction_turn(example)
    elif source_format == "oig_human_bot":
        conversations = _normalize_oig_pair(example)
    else:
        raise ValueError(f"지원하지 않는 source format: {source_format}")

    if conversations is None:
        return None

    return {"conversations": conversations}

def _conversation_hash(record):
    payload = json.dumps(record["conversations"], ensure_ascii=False, sort_keys=True)
    return hashlib.sha1(payload.encode("utf-8")).hexdigest()

def _load_single_spec(spec, seed, global_seen_hashes):
    dataset_id = spec["id"]
    split = spec["split"]
    target_n = spec["n_samples"]
    source_format = spec["format"]

    raw_ds = load_dataset(dataset_id, split=split).shuffle(seed=seed)
    dataset_columns = list(raw_ds.features.keys())

    selected = []
    scanned = 0
    dropped = 0
    duplicate = 0

    for example in raw_ds:
        scanned += 1
        normalized = normalize_example(example, source_format)
        if normalized is None:
            dropped += 1
            continue

        conv_hash = _conversation_hash(normalized)
        if conv_hash in global_seen_hashes:
            duplicate += 1
            continue

        global_seen_hashes.add(conv_hash)
        selected.append(normalized)
        if len(selected) >= target_n:
            break

    if len(selected) < target_n:
        raise RuntimeError(
            f"{dataset_id}에서 목표 샘플({target_n})을 채우지 못했습니다. "
            f"selected={len(selected)}, scanned={scanned}, dropped={dropped}, duplicate={duplicate}, "
            f"columns={dataset_columns}"
        )

    normalized_ds = Dataset.from_list(selected)
    stats = {
        "id": dataset_id,
        "split": split,
        "format": source_format,
        "target_n": target_n,
        "selected_n": len(selected),
        "scanned_n": scanned,
        "dropped_n": dropped,
        "duplicate_n": duplicate,
        "columns": dataset_columns,
    }
    return normalized_ds, stats

def build_calibration_dataset(specs, seed):
    if not specs:
        raise ValueError("CALIB_DATASETS가 비어 있습니다.")

    specs_sorted = sorted(specs, key=lambda x: x["priority"])
    subset_list = []
    stats_list = []
    global_seen_hashes = set()

    for spec in specs_sorted:
        try:
            subset, stats = _load_single_spec(spec, seed=seed, global_seen_hashes=global_seen_hashes)
            subset_list.append(subset)
            stats_list.append(stats)
        except Exception as exc:
            if spec["id"] == "beomi/KoAlpaca-v1.1a":
                print(f"[WARN] {spec['id']} 로드 실패 -> {KOALPACA_FALLBACK_ID}로 대체: {type(exc).__name__}: {exc}")
                fallback_spec = dict(spec)
                fallback_spec["id"] = KOALPACA_FALLBACK_ID
                fallback_spec["split"] = "train"
                fallback_spec["format"] = "instruction"
                subset, stats = _load_single_spec(fallback_spec, seed=seed, global_seen_hashes=global_seen_hashes)
                stats["fallback_from"] = spec["id"]
                subset_list.append(subset)
                stats_list.append(stats)
                continue
            raise

    mixed = concatenate_datasets(subset_list).shuffle(seed=seed)

    if len(mixed) != NUM_CALIBRATION_SAMPLES:
        raise RuntimeError(
            f"최종 calibration 샘플 수가 다릅니다: expected={NUM_CALIBRATION_SAMPLES}, actual={len(mixed)}"
        )

    for stats in stats_list:
        if stats["selected_n"] < min(5, stats["target_n"]):
            raise RuntimeError(f"정규화 성공 샘플이 부족합니다: {stats}")

    return mixed, stats_list

def _contains_korean(text):
    return any("가" <= ch <= "힣" for ch in text)

def _compute_ko_char_ratio(records):
    total_chars = 0
    ko_chars = 0
    for record in records:
        for turn in record["conversations"]:
            content = turn["content"]
            total_chars += len(content)
            ko_chars += sum(1 for ch in content if "가" <= ch <= "힣")
    if total_chars == 0:
        return 0.0
    return ko_chars / total_chars

ds, calibration_source_stats = build_calibration_dataset(CALIB_DATASETS, seed=CALIBRATION_SEED)

records = list(ds)
ko_char_ratio = _compute_ko_char_ratio(records)
ko_record_ratio = (
    sum(1 for item in records if _contains_korean(json.dumps(item["conversations"], ensure_ascii=False)))
    / max(len(records), 1)
)

print(f"[INFO] 혼합 캘리브레이션 샘플 수: {len(records)}")
print(f"[INFO] 한글 문자 비율: {ko_char_ratio:.4f}")
print(f"[INFO] 한글 포함 샘플 비율: {ko_record_ratio:.4f}")
print("[INFO] 소스별 통계:")
for stats in calibration_source_stats:
    print(f"  - {stats}")

def preprocess(example):
    return {
        "text": tokenizer.apply_chat_template(
            example["conversations"],
            add_generation_prompt=True,
            tokenize=False,
        )
    }

ds = ds.map(preprocess)

print("[INFO] 데이터 전처리 완료")

[INFO] 캘리브레이션 데이터 로드 중...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-21df739eb88d71(…):   0%|          | 0.00/12.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

[INFO] 혼합 캘리브레이션 샘플 수: 1024
[INFO] 한글 문자 비율: 0.0666
[INFO] 한글 포함 샘플 비율: 0.4980
[INFO] 소스별 통계:
  - {'id': 'LGAI-EXAONE/MANTA-1M', 'split': 'train', 'format': 'manta', 'target_n': 512, 'selected_n': 512, 'scanned_n': 512, 'dropped_n': 0, 'duplicate_n': 0, 'columns': ['id', 'conversations', 'complexity_label']}
  - {'id': 'nlpai-lab/kullm-v2', 'split': 'train', 'format': 'instruction', 'target_n': 256, 'selected_n': 256, 'scanned_n': 257, 'dropped_n': 1, 'duplicate_n': 0, 'columns': ['id', 'instruction', 'input', 'output']}
  - {'id': 'heegyu/OIG-small-chip2-ko', 'split': 'train', 'format': 'oig_human_bot', 'target_n': 192, 'selected_n': 192, 'scanned_n': 192, 'dropped_n': 0, 'duplicate_n': 0, 'columns': ['user', 'chip2', 'index', 'user_translated', 'chip2_translated']}
  - {'id': 'beomi/KoAlpaca-v1.1a', 'split': 'train', 'format': 'instruction', 'target_n': 64, 'selected_n': 64, 'scanned_n': 64, 'dropped_n': 0, 'duplicate_n': 0, 'columns': ['instruction', 'output', 'url']}


Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

[INFO] 데이터 전처리 완료


In [8]:
def build_gptq_modifier(scheme_name):
    return GPTQModifier(
        scheme=scheme_name,
        targets=ATTN_TARGETS,
        ignore=IGNORE,
    )


candidate_schemes = [PRIMARY_SCHEME]
if FALLBACK_SCHEME and FALLBACK_SCHEME not in candidate_schemes:
    candidate_schemes.append(FALLBACK_SCHEME)

applied_scheme = None
applied_mode = None
last_error = None

for idx, scheme_name in enumerate(candidate_schemes, start=1):
    print(
        f"[INFO] GPTQ(attention-only) start ({idx}/{len(candidate_schemes)}, "
        f"scheme={scheme_name}, samples={NUM_CALIBRATION_SAMPLES}, max_len={MAX_SEQUENCE_LENGTH})"
    )

    try:
        recipe = [build_gptq_modifier(scheme_name)]
        oneshot(
            model=model,
            dataset=ds,
            recipe=recipe,
            max_seq_length=MAX_SEQUENCE_LENGTH,
            num_calibration_samples=NUM_CALIBRATION_SAMPLES,
        )
        applied_scheme = scheme_name
        applied_mode = f"gptq_attn_only_{scheme_name.lower()}"
        print(f"[INFO] GPTQ(attention-only) success (scheme={scheme_name})")
        break
    except (ValueError, RuntimeError) as e:
        last_error = e
        print(f"[WARN] GPTQ(attention-only) failed (scheme={scheme_name}): {type(e).__name__}: {e}")

if applied_scheme is None:
    raise RuntimeError(
        f"All quantization schemes failed: {candidate_schemes}. Last error: {last_error}"
    )

print(f"[INFO] quantization complete (applied_mode={applied_mode}, applied_scheme={applied_scheme})")


[INFO] GPTQ(attention-only) start (1/2, scheme=W4A16, samples=1024, max_len=512)


Tokenizing:   0%|          | 0/1024 [00:00<?, ? examples/s]

2026-02-13T18:30:50.224497+0000 | reset | INFO - Compression lifecycle reset
2026-02-13T18:30:50.227398+0000 | from_modifiers | INFO - Creating recipe from modifiers
2026-02-13T18:30:50.266046+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2026-02-13T18:30:50.266996+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `GPTQModifier`


(1/31): Calibrating: 100%|██████████| 1024/1024 [00:06<00:00, 159.93it/s]

2026-02-13T18:31:04.371620+0000 | compress_modules | INFO - Quantizing model.layers.0.self_attn.q_proj using 1024 samples


...

2026-02-13T18:36:52.784920+0000 | compress_modules | INFO - Quantizing model.layers.29.self_attn.q_proj using 1024 samples


2026-02-13T18:36:53.999366+0000 | compress | METRIC - time 1.21s
2026-02-13T18:36:54.001504+0000 | compress | METRIC - error 2680.24
2026-02-13T18:36:54.002671+0000 | compress | METRIC - GPU 0 | usage: 2.40% | total memory: 85 GB
2026-02-13T18:36:54.003314+0000 | compress | METRIC - Compressed module size: 8.486912 MB
2026-02-13T18:36:54.004413+0000 | compress_modules | INFO - Quantizing model.layers.29.self_attn.k_proj using 1024 samples
2026-02-13T18:36:55.199883+0000 | compress | METRIC - time 1.19s
2026-02-13T18:36:55.201902+0000 | compress | METRIC - error 766.38
2026-02-13T18:36:55.202997+0000 | compress | METRIC - GPU 0 | usage: 2.40% | total memory: 85 GB
2026-02-13T18:36:55.203641+0000 | compress | METRIC - Compressed module size: 2.121728 MB
2026-02-13T18:36:55.204630+0000 | compress_modules | INFO - Quantizing model.layers.29.self_attn.v_proj using 1024 samples
2026-02-13T18:36:56.401971+0000 | compress | METRIC - time 1.20s
2026-02-13T18:36:56.403832+0000 | compress | METRI

(31/31): Propagating: 100%|██████████| 1024/1024 [00:00<00:00, 1070.00it/s]

2026-02-13T18:37:02.920003+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers


2026-02-13T18:37:02.962827+0000 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`
[INFO] GPTQ(attention-only) success (scheme=W4A16)
[INFO] quantization complete (applied_mode=gptq_attn_only_w4a16, applied_scheme=W4A16)


In [9]:
os.makedirs(OUT_DIR, exist_ok=True)

model.save_pretrained(OUT_DIR, save_compressed=True)
tokenizer.save_pretrained(OUT_DIR)

quant_recipe = {
    "timestamp_utc": datetime.now(timezone.utc).isoformat(),
    "model_id": MODEL_ID,
    "calibration_sources": CALIB_DATASETS,
    "calibration_seed": CALIBRATION_SEED,
    "calibration_benchmark_exclude": sorted(CALIBRATION_BENCHMARK_EXCLUDE),
    "calibration_source_stats": calibration_source_stats,
    "num_calibration_samples": NUM_CALIBRATION_SAMPLES,
    "max_sequence_length": MAX_SEQUENCE_LENGTH,
    "primary_scheme": PRIMARY_SCHEME,
    "fallback_scheme": FALLBACK_SCHEME,
    "applied_scheme": applied_scheme,
    "applied_mode": applied_mode,
    "targets": ATTN_TARGETS,
    "ignore": IGNORE,
    "out_dir": OUT_DIR,
}

recipe_path = Path(OUT_DIR) / "quant_recipe.json"
recipe_path.write_text(json.dumps(quant_recipe, indent=2), encoding="utf-8")

print(f"[INFO] quant recipe saved: {recipe_path}")
print(f"[INFO] 모델 저장 완료: {OUT_DIR}")


2026-02-13T18:37:03.335362+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 120it [00:00, 160.40it/s]


[INFO] quant recipe saved: /content/drive/MyDrive/LGAimers/sq_w4a16_attn_calmix1024/quant_recipe.json
[INFO] 모델 저장 완료: /content/drive/MyDrive/LGAimers/sq_w4a16_attn_calmix1024


In [10]:
zip_name = f"/content/drive/MyDrive/LGAimers/submit/{Path(OUT_DIR).name}"
zip_path = Path(zip_name)

zip_path.parent.mkdir(parents=True, exist_ok=True)  # 경로 없으면 생성

from tempfile import TemporaryDirectory
with TemporaryDirectory() as tmpdir:
    tmp_root = Path(tmpdir)
    model_dir = tmp_root / "model"
    shutil.copytree(OUT_DIR, model_dir)
    shutil.make_archive(str(zip_path), "zip", root_dir=tmp_root, base_dir="model")

print(f"[INFO] 생성 완료: {zip_name}.zip")


[INFO] 생성 완료: /content/drive/MyDrive/LGAimers/submit/sq_w4a16_attn_calmix1024.zip
